# Module 01 — Mathematical & Programming Foundations## 01-06: Linear Algebra for Machine Learning**Objective:** Build the core linear algebra toolkit used throughout ML —eigendecomposition, singular value decomposition (SVD), and low-rankapproximation — from scratch, then apply them to real data.**Prerequisites:** 01-01 (Python, NumPy & Tensor Speed), 01-02 (Advanced NumPy & PyTorch Operations)

---## Part 0 — Setup & PrerequisitesLinear algebra is the language of machine learning. Matrices representdatasets, weight matrices define neural network layers, and matrixdecompositions reveal the hidden structure in data.This notebook covers:- **Vector and matrix norms** — measuring magnitude and distance- **Matrix decompositions** — eigendecomposition, SVD, Cholesky- **Low-rank approximation** — compressing matrices while preserving information- **PCA from scratch** — the most important application of eigendecomposition- **Condition numbers** — understanding numerical stabilityWe use synthetic data, the California Housing dataset, and image datato ground every concept in a practical ML application.**Prerequisites:** 01-01, 01-02 (Advanced NumPy & PyTorch Operations)

In [ ]:
# ── Imports ──────────────────────────────────────────────────────────────────
import sys
import warnings
warnings.filterwarnings('ignore')

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from sklearn.datasets import fetch_california_housing, make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA as SklearnPCA

print(f'Python: {sys.version.split()[0]}')
print(f'PyTorch: {torch.__version__}')
print(f'NumPy: {np.__version__}')
if torch.cuda.is_available():
    print(f'CUDA: {torch.version.cuda}')
    print(f'GPU: {torch.cuda.get_device_name(0)}')

In [ ]:
# ── Reproducibility ──────────────────────────────────────────────────────────
import random

SEED = 1103
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

In [ ]:
# ── Configuration ────────────────────────────────────────────────────────────
FIGSIZE = (10, 6)
COLORS = {
    'blue': '#1E88E5',
    'red': '#E53935',
    'green': '#43A047',
    'orange': '#FF9800',
    'purple': '#9C27B0',
    'teal': '#00897B',
    'gray': '#757575',
}

### Data LoadingWe load the California Housing dataset for PCA and regression examples,and create synthetic data for geometric visualizations.

In [ ]:
# California Housing
housing = fetch_california_housing()
X_housing = housing.data.astype(np.float64)
y_housing = housing.target.astype(np.float64)
feature_names = housing.feature_names
print(f'California Housing: {X_housing.shape} ({X_housing.shape[1]} features)')
print(f'Features: {feature_names}')

# Standardize for PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_housing)
print(f'Scaled data: mean ≈ {X_scaled.mean(axis=0).round(2)}, std ≈ {X_scaled.std(axis=0).round(2)}')

# Synthetic 2D data for geometric visualization
rng = np.random.RandomState(SEED)
cov_matrix = np.array([[3.0, 1.5], [1.5, 1.0]])
X_2d = rng.multivariate_normal(mean=[0, 0], cov=cov_matrix, size=300)
print(f'\n2D synthetic data: {X_2d.shape}')
print(f'  Covariance matrix:\n{np.cov(X_2d.T).round(3)}')

---## Part 1 — Linear Algebra Foundations from ScratchWe build up from norms and inner products to the two most importantmatrix decompositions in ML: eigendecomposition and SVD.

### 1.1 Vector and Matrix NormsNorms measure the *size* of vectors and matrices. They appear everywherein ML — loss functions, regularization, gradient clipping, convergence.**Vector norms:**- $L^1$ norm: $\|\mathbf{x}\|_1 = \sum_i |x_i|$ — sparsity measure (Lasso)- $L^2$ norm: $\|\mathbf{x}\|_2 = \sqrt{\sum_i x_i^2}$ — Euclidean distance (Ridge)- $L^\infty$ norm: $\|\mathbf{x}\|_\infty = \max_i |x_i|$ — largest component**Matrix norms:**- Frobenius norm: $\|\mathbf{A}\|_F = \sqrt{\sum_{i,j} a_{ij}^2}$ — "size" of a matrix- Spectral norm: $\|\mathbf{A}\|_2 = \sigma_{\max}(\mathbf{A})$ — largest singular value

In [ ]:
def compute_vector_norms(x: np.ndarray) -> dict[str, float]:
    """Compute L1, L2, and L-infinity norms of a vector.

    Args:
        x: Input vector of shape (n,).

    Returns:
        Dictionary mapping norm names to values.
    """
    l1 = np.sum(np.abs(x))
    l2 = np.sqrt(np.sum(x ** 2))
    linf = np.max(np.abs(x))
    return {'L1': l1, 'L2': l2, 'L_inf': linf}


def compute_matrix_norms(A: np.ndarray) -> dict[str, float]:
    """Compute Frobenius and spectral norms of a matrix.

    Args:
        A: Input matrix of shape (m, n).

    Returns:
        Dictionary mapping norm names to values.
    """
    frobenius = np.sqrt(np.sum(A ** 2))
    spectral = np.max(np.linalg.svd(A, compute_uv=False))
    return {'Frobenius': frobenius, 'Spectral': spectral}


# Test with example vectors and matrices
x_test = np.array([3.0, -4.0, 1.0, 2.0])
A_test = np.array([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])

vec_norms = compute_vector_norms(x_test)
mat_norms = compute_matrix_norms(A_test)

print(f'Vector x = {x_test}')
for name, val in vec_norms.items():
    np_val = np.linalg.norm(x_test, ord={'L1': 1, 'L2': 2, 'L_inf': np.inf}[name])
    print(f'  {name}: {val:.4f} (NumPy: {np_val:.4f}) ✓' if np.isclose(val, np_val) else f'  {name}: MISMATCH')

print(f'\nMatrix A shape = {A_test.shape}')
for name, val in mat_norms.items():
    np_ord = 'fro' if name == 'Frobenius' else 2
    np_val = np.linalg.norm(A_test, ord=np_ord)
    print(f'  {name}: {val:.4f} (NumPy: {np_val:.4f}) ✓' if np.isclose(val, np_val) else f'  {name}: MISMATCH')

### 1.2 Geometric Meaning of Linear TransformationsA matrix $\mathbf{A}$ transforms vectors by stretching, rotating, or reflectingthem. Understanding this geometry is key to understanding eigenvaluesand singular values.

In [ ]:
def visualize_linear_transformation(A: np.ndarray) -> None:
    """Visualize how a 2x2 matrix transforms the unit circle.

    Args:
        A: 2x2 transformation matrix.
    """
    assert A.shape == (2, 2), f'Expected (2,2), got {A.shape}'

    theta = np.linspace(0, 2 * np.pi, 100)
    circle = np.vstack([np.cos(theta), np.sin(theta)])  # (2, 100)
    transformed = A @ circle  # (2, 100)

    # Basis vectors
    e1 = np.array([1, 0])
    e2 = np.array([0, 1])
    Ae1 = A @ e1
    Ae2 = A @ e2

    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # Before
    axes[0].plot(circle[0], circle[1], color=COLORS['blue'], linewidth=2)
    axes[0].arrow(0, 0, e1[0]*0.9, e1[1]*0.9, head_width=0.08,
                   color=COLORS['red'], linewidth=2)
    axes[0].arrow(0, 0, e2[0]*0.9, e2[1]*0.9, head_width=0.08,
                   color=COLORS['green'], linewidth=2)
    axes[0].set_title('Before: Unit Circle')
    axes[0].set_xlim(-3, 3)
    axes[0].set_ylim(-3, 3)
    axes[0].set_aspect('equal')
    axes[0].grid(True, alpha=0.3)
    axes[0].axhline(0, color='gray', linewidth=0.5)
    axes[0].axvline(0, color='gray', linewidth=0.5)

    # After
    axes[1].plot(transformed[0], transformed[1], color=COLORS['blue'], linewidth=2)
    axes[1].arrow(0, 0, Ae1[0]*0.9, Ae1[1]*0.9, head_width=0.08,
                   color=COLORS['red'], linewidth=2)
    axes[1].arrow(0, 0, Ae2[0]*0.9, Ae2[1]*0.9, head_width=0.08,
                   color=COLORS['green'], linewidth=2)
    axes[1].set_title(f'After: A · circle (det={np.linalg.det(A):.2f})')
    axes[1].set_xlim(-3, 3)
    axes[1].set_ylim(-3, 3)
    axes[1].set_aspect('equal')
    axes[1].grid(True, alpha=0.3)
    axes[1].axhline(0, color='gray', linewidth=0.5)
    axes[1].axvline(0, color='gray', linewidth=0.5)

    plt.suptitle(f'Linear Transformation: A = {A.tolist()}', fontsize=13)
    plt.tight_layout()
    plt.show()


# Stretching + rotation
A_transform = np.array([[2.0, 0.5], [0.5, 1.0]])
visualize_linear_transformation(A_transform)

### 1.3 Eigendecomposition from ScratchFor a square matrix $\mathbf{A}$, an eigenvector $\mathbf{v}$ satisfies:$$\mathbf{A}\mathbf{v} = \lambda \mathbf{v}$$where $\lambda$ is the corresponding eigenvalue. The matrix can be decomposed as:$$\mathbf{A} = \mathbf{V} \boldsymbol{\Lambda} \mathbf{V}^{-1}$$For symmetric matrices (covariance matrices!), $\mathbf{V}$ is orthogonal:$\mathbf{A} = \mathbf{V} \boldsymbol{\Lambda} \mathbf{V}^T$We implement the **power iteration** method to find the dominant eigenvector.

In [ ]:
def power_iteration(
    A: np.ndarray,
    num_iterations: int = 100,
    tolerance: float = 1e-10,
) -> tuple[float, np.ndarray]:
    """Find the dominant eigenvalue/eigenvector using power iteration.

    The power method repeatedly multiplies a random vector by A and
    normalizes. It converges to the eigenvector with the largest
    absolute eigenvalue.

    Args:
        A: Square matrix of shape (n, n).
        num_iterations: Maximum iterations.
        tolerance: Convergence threshold for eigenvalue change.

    Returns:
        Tuple of (dominant_eigenvalue, dominant_eigenvector).
    """
    n = A.shape[0]
    assert A.shape == (n, n), f'Expected square matrix, got {A.shape}'

    # Start with random vector
    rng = np.random.RandomState(SEED)
    v = rng.randn(n)
    v = v / np.linalg.norm(v)

    eigenvalue = 0.0
    for iteration in range(num_iterations):
        # Multiply by A
        Av = A @ v

        # Rayleigh quotient for eigenvalue estimate
        new_eigenvalue = v @ Av

        # Normalize
        v = Av / np.linalg.norm(Av)

        # Check convergence
        if abs(new_eigenvalue - eigenvalue) < tolerance:
            break
        eigenvalue = new_eigenvalue

    return eigenvalue, v


# Test on our covariance matrix
cov_2d = np.cov(X_2d.T)
our_eigval, our_eigvec = power_iteration(cov_2d)

# Compare with NumPy
np_eigvals, np_eigvecs = np.linalg.eigh(cov_2d)  # eigh for symmetric
# NumPy returns in ascending order; dominant is the last
np_dominant_val = np_eigvals[-1]
np_dominant_vec = np_eigvecs[:, -1]

# Eigenvectors can differ by sign; compare absolute values
sign_match = np.sign(our_eigvec[0]) == np.sign(np_dominant_vec[0])
if not sign_match:
    np_dominant_vec = -np_dominant_vec

print(f'Covariance matrix:\n{cov_2d.round(4)}')
print(f'\nPower iteration:')
print(f'  Eigenvalue: {our_eigval:.6f}')
print(f'  Eigenvector: {our_eigvec.round(6)}')
print(f'\nNumPy eigh:')
print(f'  Eigenvalue: {np_dominant_val:.6f}')
print(f'  Eigenvector: {np_dominant_vec.round(6)}')
print(f'\nMatch: eigenvalue close = {np.isclose(our_eigval, np_dominant_val)}')
print(f'Match: eigenvector close = {np.allclose(our_eigvec, np_dominant_vec, atol=1e-4)}')

Now let's implement **deflation** to find all eigenvalues/eigenvectors iteratively.The idea: after finding the dominant eigenpair $(\lambda_1, \mathbf{v}_1)$, wesubtract its contribution and repeat on the residual matrix.

In [ ]:
def eigendecomposition_power(
    A: np.ndarray,
    num_components: int | None = None,
) -> tuple[np.ndarray, np.ndarray]:
    """Compute eigendecomposition using power iteration + deflation.

    Only works well for symmetric positive semi-definite matrices
    (like covariance matrices).

    Args:
        A: Symmetric matrix of shape (n, n).
        num_components: Number of eigenvalues to compute. Defaults to n.

    Returns:
        Tuple of (eigenvalues, eigenvectors) sorted descending by eigenvalue.
        eigenvalues has shape (k,), eigenvectors has shape (n, k).
    """
    n = A.shape[0]
    if num_components is None:
        num_components = n

    eigenvalues = []
    eigenvectors = []
    residual = A.copy()

    for _ in range(num_components):
        eigval, eigvec = power_iteration(residual)
        eigenvalues.append(eigval)
        eigenvectors.append(eigvec)

        # Deflate: remove this component
        residual = residual - eigval * np.outer(eigvec, eigvec)

    return np.array(eigenvalues), np.column_stack(eigenvectors)


# Full eigendecomposition of 2D covariance
our_vals, our_vecs = eigendecomposition_power(cov_2d)

print('Our eigendecomposition (descending order):')
for i in range(len(our_vals)):
    print(f'  λ_{i+1} = {our_vals[i]:.6f}, v_{i+1} = {our_vecs[:, i].round(6)}')

print(f'\nNumPy eigenvalues (ascending): {np_eigvals.round(6)}')
print(f'Our eigenvalues (descending):  {our_vals.round(6)}')

# Verify reconstruction: A ≈ V Λ V^T
reconstructed = our_vecs @ np.diag(our_vals) @ our_vecs.T
reconstruction_error = np.linalg.norm(cov_2d - reconstructed, 'fro')
print(f'\nReconstruction error (Frobenius): {reconstruction_error:.10f}')

### 1.4 Visualizing EigenvectorsEigenvectors of the covariance matrix show the principal axes of variation.The eigenvalues tell us how much variance lies along each axis.

In [ ]:
def visualize_eigenvectors(X: np.ndarray, eigenvalues: np.ndarray, eigenvectors: np.ndarray) -> None:
    """Visualize data with eigenvectors of the covariance matrix.

    Args:
        X: Data matrix of shape (n_samples, 2).
        eigenvalues: Eigenvalues of shape (2,).
        eigenvectors: Eigenvectors of shape (2, 2), columns are eigenvectors.
    """
    fig, ax = plt.subplots(figsize=(8, 8))

    # Plot data
    ax.scatter(X[:, 0], X[:, 1], s=10, alpha=0.4, color=COLORS['blue'])

    # Plot eigenvectors (scaled by eigenvalue for visualization)
    mean = X.mean(axis=0)
    for i, (val, color, label) in enumerate([
        (eigenvalues[0], COLORS['red'], 'PC1'),
        (eigenvalues[1], COLORS['green'], 'PC2'),
    ]):
        vec = eigenvectors[:, i] * np.sqrt(val) * 2  # Scale for visibility
        ax.annotate('', xy=mean + vec, xytext=mean,
                     arrowprops=dict(arrowstyle='->', color=color, lw=3))
        ax.annotate(f'{label} (λ={val:.2f})',
                     xy=mean + vec * 1.1, fontsize=11, color=color,
                     fontweight='bold')

    ax.set_xlabel('$x_1$')
    ax.set_ylabel('$x_2$')
    ax.set_title('Data with Principal Component Directions')
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    # Variance explained
    total_var = eigenvalues.sum()
    explained = eigenvalues / total_var * 100
    print(f'Variance explained: PC1={explained[0]:.1f}%, PC2={explained[1]:.1f}%')
    print(f'Total variance: {total_var:.4f}')


visualize_eigenvectors(X_2d, our_vals, our_vecs)

### 1.5 Singular Value Decomposition (SVD)SVD generalizes eigendecomposition to **any** matrix (not just square).Every matrix $\mathbf{A} \in \mathbb{R}^{m \times n}$ can be decomposed as:$$\mathbf{A} = \mathbf{U} \boldsymbol{\Sigma} \mathbf{V}^T$$where:- $\mathbf{U} \in \mathbb{R}^{m \times m}$ — left singular vectors (orthogonal)- $\boldsymbol{\Sigma} \in \mathbb{R}^{m \times n}$ — singular values on diagonal (non-negative)- $\mathbf{V} \in \mathbb{R}^{n \times n}$ — right singular vectors (orthogonal)The singular values $\sigma_i$ are the square roots of eigenvalues of $\mathbf{A}^T\mathbf{A}$.**Why SVD matters in ML:**- PCA (most common dimensionality reduction)- Low-rank approximation (model compression)- Pseudoinverse (least squares solutions)- Matrix completion (recommendation systems)

In [ ]:
def svd_from_eigen(A: np.ndarray) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Compute SVD by eigendecomposing A^T A and A A^T.

    This demonstrates the connection between SVD and eigendecomposition.
    Not numerically stable for large matrices — use np.linalg.svd in practice.

    Args:
        A: Matrix of shape (m, n).

    Returns:
        Tuple of (U, sigma, Vt) where A ≈ U @ diag(sigma) @ Vt.
    """
    m, n = A.shape

    # Right singular vectors: eigenvectors of A^T A
    AtA = A.T @ A
    eigvals_v, V = np.linalg.eigh(AtA)

    # Sort descending
    idx = np.argsort(eigvals_v)[::-1]
    eigvals_v = eigvals_v[idx]
    V = V[:, idx]

    # Singular values = sqrt of eigenvalues (clip for numerical stability)
    sigma = np.sqrt(np.maximum(eigvals_v, 0))

    # Left singular vectors: U = A V Σ^{-1}
    rank = np.sum(sigma > 1e-10)
    U = np.zeros((m, m))
    for i in range(rank):
        U[:, i] = A @ V[:, i] / sigma[i]

    # Fill remaining columns with orthogonal complement (Gram-Schmidt)
    if rank < m:
        remaining = np.eye(m)
        for i in range(rank, m):
            vec = remaining[:, i]
            for j in range(i):
                vec = vec - (U[:, j] @ vec) * U[:, j]
            norm = np.linalg.norm(vec)
            if norm > 1e-10:
                U[:, i] = vec / norm

    return U, sigma[:min(m, n)], V.T


# Test SVD on a small matrix
A_svd = np.array([[1.0, 2.0, 3.0],
                   [4.0, 5.0, 6.0]], dtype=np.float64)

U_ours, sigma_ours, Vt_ours = svd_from_eigen(A_svd)
U_np, sigma_np, Vt_np = np.linalg.svd(A_svd, full_matrices=True)

print(f'Matrix A ({A_svd.shape}):')
print(A_svd)
print(f'\nOur singular values:  {sigma_ours.round(6)}')
print(f'NumPy singular values: {sigma_np.round(6)}')
print(f'Match: {np.allclose(sigma_ours, sigma_np, atol=1e-6)}')

# Verify reconstruction
Sigma_mat = np.zeros_like(A_svd)
np.fill_diagonal(Sigma_mat, sigma_ours)
reconstructed_svd = U_ours @ Sigma_mat @ Vt_ours
print(f'\nReconstruction error: {np.linalg.norm(A_svd - reconstructed_svd):.10f}')

### 1.6 Low-Rank Approximation (Truncated SVD)The **Eckart-Young theorem** says that the best rank-$k$ approximationof a matrix (in Frobenius norm) is obtained by keeping only the top $k$singular values:$$\mathbf{A}_k = \sum_{i=1}^{k} \sigma_i \mathbf{u}_i \mathbf{v}_i^T$$This is the mathematical foundation of:- **PCA** (keep top $k$ components)- **Model compression** (approximate weight matrices)- **Image compression** (keep top $k$ singular values of pixel matrix)- **Denoising** (noise lives in small singular values)

In [ ]:
def low_rank_approximation(
    A: np.ndarray,
    rank: int,
) -> tuple[np.ndarray, float, float]:
    """Compute rank-k approximation of a matrix using truncated SVD.

    Args:
        A: Input matrix of shape (m, n).
        rank: Number of singular values to keep.

    Returns:
        Tuple of (approximation, relative_error, compression_ratio).
    """
    U, sigma, Vt = np.linalg.svd(A, full_matrices=False)

    # Truncate to rank k
    U_k = U[:, :rank]
    sigma_k = sigma[:rank]
    Vt_k = Vt[:rank, :]

    # Reconstruct
    A_k = U_k @ np.diag(sigma_k) @ Vt_k

    # Error metrics
    relative_error = np.linalg.norm(A - A_k, 'fro') / np.linalg.norm(A, 'fro')

    # Compression: store U_k (m×k) + sigma_k (k) + Vt_k (k×n) vs m×n
    m, n = A.shape
    original_params = m * n
    compressed_params = m * rank + rank + rank * n
    compression_ratio = original_params / compressed_params

    return A_k, relative_error, compression_ratio


# Demonstrate on a random matrix
rng = np.random.RandomState(SEED)
A_demo = rng.randn(50, 30)

results = []
for k in [1, 2, 5, 10, 20, 30]:
    A_k, rel_err, comp_ratio = low_rank_approximation(A_demo, k)
    results.append({
        'Rank': k,
        'Relative Error': f'{rel_err:.4f}',
        'Error %': f'{rel_err*100:.1f}%',
        'Compression': f'{comp_ratio:.1f}×',
        'Params (original)': A_demo.size,
        'Params (compressed)': 50 * k + k + k * 30,
    })

results_df = pd.DataFrame(results)
print('=== Low-Rank Approximation Quality ===')
print(results_df.to_string(index=False))

### 1.7 Image Compression with SVDA classic demonstration of low-rank approximation: compress a grayscaleimage by keeping only the top $k$ singular values.

In [ ]:
def demonstrate_image_compression() -> None:
    """Show SVD-based image compression at different ranks."""
    # Create a synthetic grayscale image (gradient + shapes)
    rng = np.random.RandomState(SEED)
    size = 128
    image = np.zeros((size, size))

    # Gradient background
    x_grid, y_grid = np.meshgrid(np.linspace(0, 1, size), np.linspace(0, 1, size))
    image += 0.3 * x_grid + 0.2 * y_grid

    # Add circles
    for cx, cy, r, intensity in [(40, 40, 15, 0.8), (90, 60, 20, 0.6), (60, 100, 12, 0.7)]:
        mask = (x_grid * size - cx)**2 + (y_grid * size - cy)**2 < r**2
        image[mask] = intensity

    # Add noise
    image += rng.randn(size, size) * 0.05
    image = np.clip(image, 0, 1)

    # Compute SVD
    U, sigma, Vt = np.linalg.svd(image, full_matrices=False)

    # Compress at different ranks
    ranks = [1, 5, 10, 20, 50, 128]
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()

    for idx, k in enumerate(ranks):
        A_k = U[:, :k] @ np.diag(sigma[:k]) @ Vt[:k, :]
        rel_err = np.linalg.norm(image - A_k, 'fro') / np.linalg.norm(image, 'fro')
        comp_ratio = (size * size) / (size * k + k + k * size)

        axes[idx].imshow(A_k, cmap='gray', vmin=0, vmax=1)
        axes[idx].set_title(f'Rank {k} (error={rel_err:.2%}, {comp_ratio:.1f}× compression)')
        axes[idx].axis('off')

    plt.suptitle('SVD Image Compression', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

    # Singular value spectrum
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    axes[0].semilogy(sigma, 'o-', color=COLORS['blue'], markersize=3)
    axes[0].set_xlabel('Index')
    axes[0].set_ylabel('Singular Value (log scale)')
    axes[0].set_title('Singular Value Spectrum')
    axes[0].grid(True, alpha=0.3)

    cumulative_energy = np.cumsum(sigma ** 2) / np.sum(sigma ** 2)
    axes[1].plot(cumulative_energy, color=COLORS['green'], linewidth=2)
    axes[1].axhline(0.95, color=COLORS['red'], linestyle='--', label='95% energy')
    axes[1].axhline(0.99, color=COLORS['orange'], linestyle='--', label='99% energy')
    k_95 = np.searchsorted(cumulative_energy, 0.95) + 1
    k_99 = np.searchsorted(cumulative_energy, 0.99) + 1
    axes[1].axvline(k_95, color=COLORS['red'], linestyle=':', alpha=0.5)
    axes[1].axvline(k_99, color=COLORS['orange'], linestyle=':', alpha=0.5)
    axes[1].set_xlabel('Number of Singular Values')
    axes[1].set_ylabel('Cumulative Energy')
    axes[1].set_title(f'Cumulative Energy (95% at k={k_95}, 99% at k={k_99})')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()


demonstrate_image_compression()

### 1.8 Special Matrices in MLCertain matrix types appear repeatedly in ML and have efficient decompositions:- **Symmetric positive definite (SPD):** Covariance matrices, kernel matrices,  Hessians. All eigenvalues are positive. Has Cholesky decomposition $\mathbf{A} = \mathbf{L}\mathbf{L}^T$.- **Orthogonal:** $\mathbf{Q}^T\mathbf{Q} = \mathbf{I}$. Rotation matrices, eigenvectors of  symmetric matrices. Preserves lengths and angles.- **Diagonal:** Only diagonal entries are nonzero. Scaling operations.- **Sparse:** Most entries are zero. Adjacency matrices, bag-of-words.

In [ ]:
def demonstrate_special_matrices() -> None:
    """Demonstrate special matrix types and the Cholesky decomposition."""
    rng = np.random.RandomState(SEED)

    # Build a symmetric positive definite matrix
    B = rng.randn(4, 4)
    spd = B.T @ B + 0.1 * np.eye(4)  # Guaranteed SPD

    # Cholesky decomposition: A = L L^T
    L = np.linalg.cholesky(spd)
    reconstructed = L @ L.T

    print('=== Symmetric Positive Definite Matrix ===')
    print(f'Eigenvalues: {np.linalg.eigvalsh(spd).round(4)}')
    print(f'All positive: {np.all(np.linalg.eigvalsh(spd) > 0)}')
    print()

    print('=== Cholesky Decomposition (A = L L^T) ===')
    print(f'L (lower triangular):')
    print(L.round(4))
    print(f'\nL is lower triangular: {np.allclose(L, np.tril(L))}')
    print(f'Reconstruction error: {np.linalg.norm(spd - reconstructed):.2e}')
    print()

    # Cholesky is 2x faster than general eigendecomposition
    large_B = rng.randn(200, 200)
    large_spd = large_B.T @ large_B + np.eye(200)

    start = time.perf_counter()
    for _ in range(100):
        np.linalg.cholesky(large_spd)
    chol_time = time.perf_counter() - start

    start = time.perf_counter()
    for _ in range(100):
        np.linalg.eigh(large_spd)
    eigh_time = time.perf_counter() - start

    print(f'200×200 SPD matrix (100 runs):')
    print(f'  Cholesky: {chol_time*1000:.1f}ms')
    print(f'  Eigendecomposition: {eigh_time*1000:.1f}ms')
    print(f'  Cholesky speedup: {eigh_time/chol_time:.1f}×')
    print()

    # Orthogonal matrix check
    Q, _ = np.linalg.qr(rng.randn(4, 4))
    print('=== Orthogonal Matrix ===')
    print(f'Q^T Q ≈ I: {np.allclose(Q.T @ Q, np.eye(4))}')
    print(f'det(Q) = {np.linalg.det(Q):.4f} (±1 for orthogonal)')
    print(f'Preserves norm: |Qx|/|x| = {np.linalg.norm(Q @ np.ones(4)) / np.linalg.norm(np.ones(4)):.6f}')


demonstrate_special_matrices()

### 1.9 Orthogonal ProjectionsProjection is the core operation behind PCA, least squares, and attention.The projection of $\mathbf{b}$ onto the column space of $\mathbf{A}$ is:$$\text{proj}_{\mathbf{A}}(\mathbf{b}) = \mathbf{A}(\mathbf{A}^T\mathbf{A})^{-1}\mathbf{A}^T \mathbf{b}$$The **projection matrix** $\mathbf{P} = \mathbf{A}(\mathbf{A}^T\mathbf{A})^{-1}\mathbf{A}^T$satisfies $\mathbf{P}^2 = \mathbf{P}$ (idempotent) and $\mathbf{P}^T = \mathbf{P}$ (symmetric).

In [ ]:
def demonstrate_projections() -> None:
    """Demonstrate orthogonal projection and its properties."""
    # Project a 3D point onto a 2D plane
    A = np.array([[1.0, 0.0], [0.0, 1.0], [0.0, 0.0]])  # xy-plane basis
    b = np.array([3.0, 4.0, 5.0])

    # Projection matrix
    P = A @ np.linalg.inv(A.T @ A) @ A.T
    proj_b = P @ b
    residual = b - proj_b

    print('=== Orthogonal Projection ===')
    print(f'Point b = {b}')
    print(f'Projection onto xy-plane: {proj_b}')
    print(f'Residual (perpendicular): {residual}')
    print(f'Residual ⊥ projection: dot = {proj_b @ residual:.10f}')
    print()

    # Properties of projection matrix
    print('Projection matrix properties:')
    print(f'  Idempotent (P² = P): {np.allclose(P @ P, P)}')
    print(f'  Symmetric (P^T = P): {np.allclose(P.T, P)}')
    print(f'  Rank = {np.linalg.matrix_rank(P)} (dimension of subspace)')
    print()

    # Connection to PCA: projecting data onto top-k eigenvectors
    # is a special case of orthogonal projection
    V_k = our_vecs[:, :1]  # Top eigenvector from earlier
    P_pca = V_k @ V_k.T   # Rank-1 projection
    X_proj_pca = (P_pca @ X_2d.T).T

    print('PCA as projection:')
    print(f'  Projection matrix rank: {np.linalg.matrix_rank(P_pca)}')
    print(f'  P² = P: {np.allclose(P_pca @ P_pca, P_pca)}')
    print(f'  Projected data shape: {X_proj_pca.shape}')
    recon_error = np.linalg.norm(X_2d - X_proj_pca, 'fro') / np.linalg.norm(X_2d, 'fro')
    print(f'  Reconstruction error: {recon_error:.4f}')


demonstrate_projections()

---## Part 2 — Putting It All Together: PCA from ScratchPrincipal Component Analysis (PCA) is the most important application ofeigendecomposition / SVD in ML. It finds the directions of maximum varianceand projects data onto them for dimensionality reduction.

In [ ]:
class PCAFromScratch:
    """Principal Component Analysis implemented from scratch using SVD.

    PCA finds orthogonal directions that maximize variance and projects
    data onto the top-k of those directions.

    Attributes:
        n_components: Number of principal components to keep.
        components: Principal component directions of shape (k, n_features).
        explained_variance: Variance along each component.
        explained_variance_ratio: Fraction of total variance per component.
        mean: Feature means used for centering.
    """

    def __init__(self, n_components: int = 2) -> None:
        """Initialize PCA.

        Args:
            n_components: Number of components to keep.
        """
        self.n_components = n_components
        self.components: np.ndarray | None = None
        self.explained_variance: np.ndarray | None = None
        self.explained_variance_ratio: np.ndarray | None = None
        self.mean: np.ndarray | None = None

    def fit(self, X: np.ndarray) -> 'PCAFromScratch':
        """Fit PCA by computing SVD of the centered data.

        Args:
            X: Data matrix of shape (n_samples, n_features).

        Returns:
            Self for method chaining.
        """
        n_samples = X.shape[0]

        # Step 1: Center the data
        self.mean = X.mean(axis=0)
        X_centered = X - self.mean

        # Step 2: SVD of centered data
        # X = U Σ V^T → covariance ∝ V Σ² V^T
        U, sigma, Vt = np.linalg.svd(X_centered, full_matrices=False)

        # Step 3: Top-k components
        self.components = Vt[:self.n_components]

        # Step 4: Explained variance
        self.explained_variance = (sigma[:self.n_components] ** 2) / (n_samples - 1)
        total_variance = np.sum(sigma ** 2) / (n_samples - 1)
        self.explained_variance_ratio = self.explained_variance / total_variance

        return self

    def transform(self, X: np.ndarray) -> np.ndarray:
        """Project data onto principal components.

        Args:
            X: Data matrix of shape (n_samples, n_features).

        Returns:
            Projected data of shape (n_samples, n_components).
        """
        assert self.components is not None, 'Must call fit() first'
        X_centered = X - self.mean
        return X_centered @ self.components.T

    def fit_transform(self, X: np.ndarray) -> np.ndarray:
        """Fit and transform in one step.

        Args:
            X: Data matrix of shape (n_samples, n_features).

        Returns:
            Projected data of shape (n_samples, n_components).
        """
        return self.fit(X).transform(X)

    def inverse_transform(self, X_projected: np.ndarray) -> np.ndarray:
        """Reconstruct data from principal component space.

        Args:
            X_projected: Projected data of shape (n_samples, n_components).

        Returns:
            Reconstructed data of shape (n_samples, n_features).
        """
        assert self.components is not None, 'Must call fit() first'
        return X_projected @ self.components + self.mean


# Sanity check on 2D data
pca_2d = PCAFromScratch(n_components=1)
X_2d_proj = pca_2d.fit_transform(X_2d)
X_2d_recon = pca_2d.inverse_transform(X_2d_proj)

assert X_2d_proj.shape == (300, 1), f'Expected (300, 1), got {X_2d_proj.shape}'
print(f'2D data: {X_2d.shape} → projected: {X_2d_proj.shape} → reconstructed: {X_2d_recon.shape}')
print(f'Variance explained: {pca_2d.explained_variance_ratio[0]:.4f} ({pca_2d.explained_variance_ratio[0]*100:.1f}%)')
print(f'Reconstruction error: {np.linalg.norm(X_2d - X_2d_recon):.4f}')

Let's visualize the PCA projection and reconstruction on our 2D data.

In [ ]:
def visualize_pca_2d(X: np.ndarray, X_recon: np.ndarray, pca: PCAFromScratch) -> None:
    """Visualize PCA projection and reconstruction on 2D data.

    Args:
        X: Original data of shape (n, 2).
        X_recon: Reconstructed data of shape (n, 2).
        pca: Fitted PCAFromScratch instance.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # Left: original + PC direction + projections
    axes[0].scatter(X[:, 0], X[:, 1], s=10, alpha=0.4, color=COLORS['blue'], label='Original')
    axes[0].scatter(X_recon[:, 0], X_recon[:, 1], s=10, alpha=0.4,
                     color=COLORS['red'], label='Projected onto PC1')

    # Draw lines from original to projection
    for i in range(0, len(X), 10):
        axes[0].plot([X[i, 0], X_recon[i, 0]], [X[i, 1], X_recon[i, 1]],
                      color=COLORS['gray'], alpha=0.2, linewidth=0.5)

    # Draw PC direction
    mean = pca.mean
    pc = pca.components[0]
    scale = 4
    axes[0].annotate('', xy=mean + pc * scale, xytext=mean - pc * scale,
                      arrowprops=dict(arrowstyle='->', color=COLORS['red'], lw=2))

    axes[0].set_xlabel('$x_1$')
    axes[0].set_ylabel('$x_2$')
    axes[0].set_title('PCA: Projection onto 1st Principal Component')
    axes[0].legend()
    axes[0].set_aspect('equal')
    axes[0].grid(True, alpha=0.3)

    # Right: reconstruction error distribution
    errors = np.linalg.norm(X - X_recon, axis=1)
    axes[1].hist(errors, bins=30, color=COLORS['purple'], alpha=0.7, edgecolor='white')
    axes[1].axvline(errors.mean(), color=COLORS['red'], linestyle='--',
                     label=f'Mean error: {errors.mean():.3f}')
    axes[1].set_xlabel('Reconstruction Error')
    axes[1].set_ylabel('Count')
    axes[1].set_title('Reconstruction Error Distribution')
    axes[1].legend()

    plt.tight_layout()
    plt.show()


visualize_pca_2d(X_2d, X_2d_recon, pca_2d)

---## Part 3 — Application: PCA on California HousingLet's apply our PCA implementation to a real dataset with 8 featuresand compare against sklearn's implementation.

In [ ]:
def apply_pca_housing() -> None:
    """Apply PCA to California Housing dataset and analyze results."""
    # Our PCA
    pca_ours = PCAFromScratch(n_components=8)
    X_proj_ours = pca_ours.fit_transform(X_scaled)

    # sklearn PCA
    pca_sklearn = SklearnPCA(n_components=8)
    X_proj_sklearn = pca_sklearn.fit_transform(X_scaled)

    # Compare explained variance ratios
    comparison = pd.DataFrame({
        'Component': [f'PC{i+1}' for i in range(8)],
        'Ours': pca_ours.explained_variance_ratio.round(6),
        'sklearn': pca_sklearn.explained_variance_ratio_.round(6),
        'Match': np.isclose(pca_ours.explained_variance_ratio,
                            pca_sklearn.explained_variance_ratio_, atol=1e-4),
    })
    print('=== Explained Variance Ratio Comparison ===')
    print(comparison.to_string(index=False))
    print()

    # Cumulative variance plot
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Scree plot
    axes[0].bar(range(1, 9), pca_ours.explained_variance_ratio * 100,
                color=COLORS['blue'], alpha=0.7)
    axes[0].set_xlabel('Principal Component')
    axes[0].set_ylabel('Explained Variance (%)')
    axes[0].set_title('Scree Plot')
    axes[0].set_xticks(range(1, 9))
    for i, v in enumerate(pca_ours.explained_variance_ratio):
        axes[0].text(i + 1, v * 100 + 0.5, f'{v*100:.1f}%', ha='center', fontsize=8)

    # Cumulative
    cumvar = np.cumsum(pca_ours.explained_variance_ratio)
    axes[1].plot(range(1, 9), cumvar * 100, 'o-', color=COLORS['green'], linewidth=2)
    axes[1].axhline(95, color=COLORS['red'], linestyle='--', label='95% threshold')
    k_95 = np.searchsorted(cumvar, 0.95) + 1
    axes[1].axvline(k_95, color=COLORS['red'], linestyle=':', alpha=0.5)
    axes[1].set_xlabel('Number of Components')
    axes[1].set_ylabel('Cumulative Explained Variance (%)')
    axes[1].set_title(f'Cumulative Variance (95% at k={k_95})')
    axes[1].set_xticks(range(1, 9))
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    print(f'Components needed for 95% variance: {k_95} (out of 8)')
    print(f'Dimensionality reduction: {8} → {k_95} ({k_95/8*100:.0f}% of original features)')


apply_pca_housing()

### 3.2 PCA for Visualization: 2D ProjectionOne of PCA's most common uses is projecting high-dimensional data to 2Dfor visualization. Let's visualize the housing data colored by price.

In [ ]:
def visualize_pca_projection() -> None:
    """Project California Housing to 2D with PCA and visualize."""
    pca_2comp = PCAFromScratch(n_components=2)
    X_2d_housing = pca_2comp.fit_transform(X_scaled)
    assert X_2d_housing.shape == (len(X_scaled), 2), f'Expected (n, 2), got {X_2d_housing.shape}'

    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # Colored by target (house price)
    sc = axes[0].scatter(
        X_2d_housing[:, 0], X_2d_housing[:, 1],
        c=y_housing, cmap='viridis', s=3, alpha=0.4,
    )
    axes[0].set_xlabel(f'PC1 ({pca_2comp.explained_variance_ratio[0]*100:.1f}%)')
    axes[0].set_ylabel(f'PC2 ({pca_2comp.explained_variance_ratio[1]*100:.1f}%)')
    axes[0].set_title('PCA Projection (color = house price)')
    plt.colorbar(sc, ax=axes[0], shrink=0.8, label='Median House Value')

    # Component loadings
    loadings = pca_2comp.components.T  # (n_features, n_components)
    for i, name in enumerate(feature_names):
        axes[1].arrow(0, 0, loadings[i, 0], loadings[i, 1],
                       head_width=0.03, color=COLORS['blue'], alpha=0.7)
        axes[1].text(loadings[i, 0] * 1.15, loadings[i, 1] * 1.15,
                      name, fontsize=9, ha='center')

    # Draw unit circle for reference
    theta = np.linspace(0, 2 * np.pi, 100)
    axes[1].plot(np.cos(theta), np.sin(theta), '--', color=COLORS['gray'], alpha=0.3)
    axes[1].set_xlabel('PC1 Loading')
    axes[1].set_ylabel('PC2 Loading')
    axes[1].set_title('Feature Loadings (Biplot)')
    axes[1].set_aspect('equal')
    axes[1].grid(True, alpha=0.3)
    axes[1].set_xlim(-1.2, 1.2)
    axes[1].set_ylim(-1.2, 1.2)

    plt.tight_layout()
    plt.show()


visualize_pca_projection()

### 3.3 PCA for Noise ReductionLow-rank approximation can denoise data. The signal lives in the topprincipal components, while noise spreads across all components.

In [ ]:
def demonstrate_pca_denoising() -> None:
    """Show PCA denoising on synthetic data."""
    rng = np.random.RandomState(SEED)

    # Create clean low-rank data
    n_samples = 200
    true_dim = 3
    n_features = 10

    # Generate from a 3D subspace
    latent = rng.randn(n_samples, true_dim)
    mixing = rng.randn(true_dim, n_features)
    X_clean = latent @ mixing

    # Add noise
    noise_level = 1.0
    noise = rng.randn(n_samples, n_features) * noise_level
    X_noisy = X_clean + noise

    # Denoise with PCA at different ranks
    pca_full = PCAFromScratch(n_components=n_features)
    pca_full.fit(X_noisy)

    errors = []
    for k in range(1, n_features + 1):
        pca_k = PCAFromScratch(n_components=k)
        X_proj = pca_k.fit_transform(X_noisy)
        X_recon = pca_k.inverse_transform(X_proj)
        error = np.linalg.norm(X_clean - X_recon, 'fro') / np.linalg.norm(X_clean, 'fro')
        errors.append(error)

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Denoising error vs rank
    axes[0].plot(range(1, n_features + 1), errors, 'o-', color=COLORS['blue'], linewidth=2)
    axes[0].axvline(true_dim, color=COLORS['red'], linestyle='--',
                     label=f'True dimension = {true_dim}')
    axes[0].set_xlabel('Number of Components')
    axes[0].set_ylabel('Relative Error vs Clean Data')
    axes[0].set_title('PCA Denoising: Error vs Components')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    best_k = np.argmin(errors) + 1
    axes[0].annotate(f'Best: k={best_k}', xy=(best_k, errors[best_k - 1]),
                      xytext=(best_k + 2, errors[best_k - 1] + 0.1),
                      arrowprops=dict(arrowstyle='->', color=COLORS['green']),
                      fontsize=10, color=COLORS['green'])

    # Explained variance (scree plot)
    axes[1].bar(range(1, n_features + 1), pca_full.explained_variance_ratio * 100,
                color=COLORS['purple'], alpha=0.7)
    axes[1].axvline(true_dim + 0.5, color=COLORS['red'], linestyle='--',
                     label='Signal / Noise boundary')
    axes[1].set_xlabel('Principal Component')
    axes[1].set_ylabel('Explained Variance (%)')
    axes[1].set_title('Scree Plot (gap reveals true dimensionality)')
    axes[1].legend()

    plt.tight_layout()
    plt.show()

    print(f'True dimensionality: {true_dim}')
    print(f'Best denoising rank: {best_k}')
    print(f'Variance in top {true_dim} components: '
          f'{pca_full.explained_variance_ratio[:true_dim].sum()*100:.1f}%')


demonstrate_pca_denoising()

---## Part 4 — Evaluation & AnalysisLet's evaluate our implementations, analyze numerical stability, andbenchmark performance.

### 4.1 Numerical Stability: Condition NumbersThe **condition number** $\kappa(\mathbf{A}) = \sigma_{\max} / \sigma_{\min}$measures how sensitive a matrix computation is to small perturbations.Large condition numbers mean numerical instability.

In [ ]:
def analyze_condition_numbers() -> None:
    """Demonstrate condition numbers and their effect on computations."""
    rng = np.random.RandomState(SEED)

    # Well-conditioned matrix
    A_good = rng.randn(5, 5)
    A_good = A_good.T @ A_good + 0.1 * np.eye(5)  # Positive definite

    # Ill-conditioned matrix (nearly singular)
    A_bad = A_good.copy()
    A_bad[4, :] = A_bad[3, :] + 1e-10 * rng.randn(5)  # Nearly duplicate row
    A_bad = A_bad.T @ A_bad

    kappa_good = np.linalg.cond(A_good)
    kappa_bad = np.linalg.cond(A_bad)

    print('=== Condition Number Analysis ===')
    print(f'Well-conditioned matrix: κ = {kappa_good:.2f}')
    print(f'Ill-conditioned matrix:  κ = {kappa_bad:.2e}')
    print()

    # Show impact: solving Ax = b with small perturbation
    x_true = np.ones(5)
    b_good = A_good @ x_true
    b_bad = A_bad @ x_true

    # Add tiny perturbation to b
    perturbation = rng.randn(5) * 1e-10

    x_good = np.linalg.solve(A_good, b_good + perturbation)
    x_bad = np.linalg.solve(A_bad, b_bad + perturbation)

    error_good = np.linalg.norm(x_good - x_true) / np.linalg.norm(x_true)
    error_bad = np.linalg.norm(x_bad - x_true) / np.linalg.norm(x_true)

    print(f'Effect of 1e-10 perturbation on solution:')
    print(f'  Well-conditioned: relative error = {error_good:.2e}')
    print(f'  Ill-conditioned:  relative error = {error_bad:.2e}')
    print()

    # Condition number reference table
    ref = pd.DataFrame({
        'Condition Number': ['κ ≈ 1', 'κ ≈ 10²', 'κ ≈ 10⁶', 'κ > 10¹⁰'],
        'Stability': ['Excellent', 'Good', 'Concerning', 'Unstable'],
        'ML Example': [
            'Orthogonal features',
            'Mildly correlated features',
            'Highly correlated features',
            'Multicollinear / near-singular',
        ],
        'Action': [
            'No issues',
            'Normal',
            'Consider regularization or PCA',
            'Must regularize or remove features',
        ],
    })
    print('=== Condition Number Reference ===')
    print(ref.to_string(index=False))


analyze_condition_numbers()

### 4.2 PCA Impact on Downstream TasksLet's measure how PCA dimensionality reduction affects prediction qualityusing a simple linear regression.

In [ ]:
def evaluate_pca_downstream() -> None:
    """Evaluate PCA's impact on downstream regression performance."""
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error, r2_score

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_housing, test_size=0.2, random_state=SEED,
    )

    results = []

    # Baseline: all features
    model_full = LinearRegression().fit(X_train, y_train)
    y_pred_full = model_full.predict(X_test)
    results.append({
        'Components': 'All (8)',
        'R²': r2_score(y_test, y_pred_full),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_full)),
        'Features': 8,
    })

    # PCA at different component counts
    for k in [1, 2, 3, 4, 5, 6, 7]:
        pca_k = PCAFromScratch(n_components=k)
        X_train_pca = pca_k.fit_transform(X_train)
        X_test_pca = pca_k.transform(X_test)

        model_k = LinearRegression().fit(X_train_pca, y_train)
        y_pred_k = model_k.predict(X_test_pca)
        results.append({
            'Components': f'PCA-{k}',
            'R²': r2_score(y_test, y_pred_k),
            'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_k)),
            'Features': k,
        })

    results_df = pd.DataFrame(results)
    results_df['R²'] = results_df['R²'].round(4)
    results_df['RMSE'] = results_df['RMSE'].round(4)
    print('=== PCA Impact on Linear Regression ===')
    print(results_df.to_string(index=False))
    print()

    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    components = list(range(1, 8)) + [8]
    r2_scores = results_df['R²'].values
    rmse_scores = results_df['RMSE'].values

    axes[0].plot(components, r2_scores, 'o-', color=COLORS['blue'], linewidth=2)
    axes[0].axhline(r2_scores[-1], color=COLORS['red'], linestyle='--',
                     alpha=0.5, label=f'Full model: {r2_scores[-1]:.4f}')
    axes[0].set_xlabel('Number of Components')
    axes[0].set_ylabel('R² Score')
    axes[0].set_title('R² vs PCA Components')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    axes[1].plot(components, rmse_scores, 'o-', color=COLORS['green'], linewidth=2)
    axes[1].set_xlabel('Number of Components')
    axes[1].set_ylabel('RMSE')
    axes[1].set_title('RMSE vs PCA Components')
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()


evaluate_pca_downstream()

### 4.3 Performance BenchmarksLet's compare our implementations against NumPy/sklearn on differentmatrix sizes.

In [ ]:
def benchmark_decompositions() -> None:
    """Benchmark our implementations vs library implementations."""
    rng = np.random.RandomState(SEED)
    results = []

    for n in [50, 100, 500, 1000]:
        X = rng.randn(n, min(n, 50))
        cov = X.T @ X / n

        # Our eigendecomposition (power iteration)
        start = time.perf_counter()
        eigendecomposition_power(cov, num_components=min(5, cov.shape[0]))
        our_eig_time = time.perf_counter() - start

        # NumPy eigendecomposition
        start = time.perf_counter()
        np.linalg.eigh(cov)
        np_eig_time = time.perf_counter() - start

        # Our PCA
        start = time.perf_counter()
        pca_ours = PCAFromScratch(n_components=5)
        pca_ours.fit_transform(X)
        our_pca_time = time.perf_counter() - start

        # sklearn PCA
        start = time.perf_counter()
        pca_sk = SklearnPCA(n_components=5)
        pca_sk.fit_transform(X)
        sk_pca_time = time.perf_counter() - start

        results.append({
            'Matrix Size': f'{n}×{min(n, 50)}',
            'Our Eigen': f'{our_eig_time*1000:.2f}ms',
            'NumPy Eigen': f'{np_eig_time*1000:.2f}ms',
            'Our PCA': f'{our_pca_time*1000:.2f}ms',
            'sklearn PCA': f'{sk_pca_time*1000:.2f}ms',
        })

    results_df = pd.DataFrame(results)
    print('=== Decomposition Benchmarks ===')
    print(results_df.to_string(index=False))
    print()
    print('NumPy uses LAPACK (Fortran) — optimized for decades.')
    print('Our implementations are for learning, not production.')


benchmark_decompositions()

### 4.4 Error Analysis: When PCA FailsPCA assumes linear relationships and is sensitive to outliers. Let'sdemonstrate cases where PCA doesn't capture the true structure.

In [ ]:
def demonstrate_pca_limitations() -> None:
    """Show cases where PCA fails to capture data structure."""
    rng = np.random.RandomState(SEED)

    fig, axes = plt.subplots(2, 3, figsize=(16, 10))

    # Case 1: PCA works well — elliptical data
    X_ellipse = rng.multivariate_normal([0, 0], [[3, 1.5], [1.5, 1]], 300)
    pca_e = PCAFromScratch(n_components=1)
    X_proj = pca_e.fit_transform(X_ellipse)
    X_recon = pca_e.inverse_transform(X_proj)
    axes[0, 0].scatter(X_ellipse[:, 0], X_ellipse[:, 1], s=10, alpha=0.5, color=COLORS['blue'])
    axes[0, 0].scatter(X_recon[:, 0], X_recon[:, 1], s=10, alpha=0.5, color=COLORS['red'])
    axes[0, 0].set_title(f'Elliptical: PC1 captures {pca_e.explained_variance_ratio[0]*100:.0f}% ✓')
    axes[0, 0].set_aspect('equal')

    # Case 2: PCA fails — circular data (no direction is better)
    theta = rng.uniform(0, 2 * np.pi, 300)
    r = 2 + rng.randn(300) * 0.3
    X_circle = np.column_stack([r * np.cos(theta), r * np.sin(theta)])
    pca_c = PCAFromScratch(n_components=1)
    X_proj_c = pca_c.fit_transform(X_circle)
    X_recon_c = pca_c.inverse_transform(X_proj_c)
    axes[0, 1].scatter(X_circle[:, 0], X_circle[:, 1], s=10, alpha=0.5, color=COLORS['blue'])
    axes[0, 1].scatter(X_recon_c[:, 0], X_recon_c[:, 1], s=10, alpha=0.5, color=COLORS['red'])
    axes[0, 1].set_title(f'Circular: PC1 captures {pca_c.explained_variance_ratio[0]*100:.0f}% ✗')
    axes[0, 1].set_aspect('equal')

    # Case 3: PCA fails — moon-shaped data
    from sklearn.datasets import make_moons
    X_moon, _ = make_moons(n_samples=300, noise=0.1, random_state=SEED)
    pca_m = PCAFromScratch(n_components=1)
    X_proj_m = pca_m.fit_transform(X_moon)
    X_recon_m = pca_m.inverse_transform(X_proj_m)
    axes[0, 2].scatter(X_moon[:, 0], X_moon[:, 1], s=10, alpha=0.5, color=COLORS['blue'])
    axes[0, 2].scatter(X_recon_m[:, 0], X_recon_m[:, 1], s=10, alpha=0.5, color=COLORS['red'])
    axes[0, 2].set_title(f'Moons: PC1 captures {pca_m.explained_variance_ratio[0]*100:.0f}% (misleading) ✗')
    axes[0, 2].set_aspect('equal')

    # Case 4: Outlier sensitivity
    X_clean = rng.multivariate_normal([0, 0], [[2, 1], [1, 1]], 300)
    X_outlier = X_clean.copy()
    X_outlier[:5] = rng.uniform(8, 12, (5, 2))  # Add 5 extreme outliers

    pca_clean = PCAFromScratch(n_components=2)
    pca_clean.fit(X_clean)
    pca_outlier = PCAFromScratch(n_components=2)
    pca_outlier.fit(X_outlier)

    axes[1, 0].scatter(X_clean[:, 0], X_clean[:, 1], s=10, alpha=0.5, color=COLORS['blue'])
    pc_clean = pca_clean.components[0]
    axes[1, 0].annotate('', xy=pc_clean * 3, xytext=-pc_clean * 3,
                          arrowprops=dict(arrowstyle='->', color=COLORS['red'], lw=2))
    axes[1, 0].set_title('Clean Data: PC1 direction')
    axes[1, 0].set_aspect('equal')

    axes[1, 1].scatter(X_outlier[:, 0], X_outlier[:, 1], s=10, alpha=0.5, color=COLORS['blue'])
    axes[1, 1].scatter(X_outlier[:5, 0], X_outlier[:5, 1], s=50, color=COLORS['red'],
                         marker='x', linewidths=2, label='Outliers')
    pc_out = pca_outlier.components[0]
    axes[1, 1].annotate('', xy=pc_out * 3, xytext=-pc_out * 3,
                          arrowprops=dict(arrowstyle='->', color=COLORS['red'], lw=2))
    axes[1, 1].set_title('With Outliers: PC1 direction shifted!')
    axes[1, 1].legend()
    axes[1, 1].set_aspect('equal')

    # Angle difference
    angle = np.degrees(np.arccos(np.clip(np.abs(pc_clean @ pc_out), 0, 1)))
    axes[1, 2].bar(['Clean', 'With Outliers'], [0, angle],
                    color=[COLORS['green'], COLORS['red']], alpha=0.7)
    axes[1, 2].set_ylabel('PC1 Direction Shift (degrees)')
    axes[1, 2].set_title(f'Outlier Impact: {angle:.1f}° rotation')

    for ax in axes.ravel():
        ax.grid(True, alpha=0.2)

    plt.suptitle('When PCA Fails', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

    print('PCA limitations:')
    limitations = pd.DataFrame({
        'Limitation': [
            'Assumes linear structure',
            'Sensitive to outliers',
            'Sensitive to feature scaling',
            'Variance ≠ importance',
        ],
        'When It Matters': [
            'Non-linear manifolds (circles, moons)',
            'Extreme values pull PC directions',
            'Features with different units/scales',
            'High-variance noise vs low-variance signal',
        ],
        'Solution': [
            'Kernel PCA, t-SNE, UMAP (Module 3)',
            'Robust PCA, remove outliers first',
            'Always standardize before PCA',
            'Domain knowledge, supervised methods',
        ],
    })
    print(limitations.to_string(index=False))


demonstrate_pca_limitations()

### 4.5 Linear Algebra Cheat Sheet for MLA comprehensive reference of the linear algebra operations used throughoutthis course.

In [ ]:
def build_linalg_reference() -> None:
    """Create a linear algebra operations reference for the course."""
    reference = pd.DataFrame({
        'Operation': [
            'Matrix multiply', 'Transpose', 'Inverse',
            'Determinant', 'Trace', 'Eigendecomposition',
            'SVD', 'Pseudoinverse', 'Condition number',
            'Frobenius norm', 'Rank',
        ],
        'NumPy': [
            'A @ B', 'A.T', 'np.linalg.inv(A)',
            'np.linalg.det(A)', 'np.trace(A)', 'np.linalg.eigh(A)',
            'np.linalg.svd(A)', 'np.linalg.pinv(A)', 'np.linalg.cond(A)',
            'np.linalg.norm(A, "fro")', 'np.linalg.matrix_rank(A)',
        ],
        'PyTorch': [
            'A @ B', 'A.T / A.mT', 'torch.linalg.inv(A)',
            'torch.linalg.det(A)', 'torch.trace(A)', 'torch.linalg.eigh(A)',
            'torch.linalg.svd(A)', 'torch.linalg.pinv(A)', 'torch.linalg.cond(A)',
            'torch.linalg.norm(A)', 'torch.linalg.matrix_rank(A)',
        ],
        'Used In': [
            'Neural nets, attention', 'Gradients, covariance',
            'Analytical solutions', 'Volume change',
            'Regularization', 'PCA, spectral methods',
            'PCA, compression', 'Least squares',
            'Numerical stability', 'Matrix complexity',
            'Intrinsic dimensionality',
        ],
    })
    print('=== Linear Algebra for ML — Reference ===')
    print(reference.to_string(index=False))


build_linalg_reference()

---## Part 5 — Summary & Lessons Learned### Key Takeaways1. **Matrix decompositions reveal hidden structure.** Eigendecomposition shows   the principal axes of variation in symmetric matrices (like covariance).   SVD generalizes this to any matrix.2. **SVD is the Swiss Army knife of linear algebra.** It enables PCA,   low-rank approximation, pseudoinverses, and noise reduction — all   from one decomposition.3. **PCA = SVD of centered data.** Project onto the top-$k$ right singular   vectors to reduce dimensionality while preserving maximum variance.   Always standardize features before PCA.4. **Low-rank approximation is optimal.** The Eckart-Young theorem guarantees   that truncated SVD gives the best rank-$k$ approximation in Frobenius norm.   This powers compression, denoising, and dimensionality reduction.5. **Check condition numbers for stability.** Large condition numbers   ($\kappa > 10^6$) signal that computations may be numerically unstable.   Regularization (adding $\lambda \mathbf{I}$) or PCA can help.### What's Next→ **01-07 (Probability & Statistics for ML)** covers distributions, MLE,  MAP estimation, and Bayes' theorem — the probabilistic foundation for  understanding loss functions and generalization.### Going Further- [Linear Algebra Done Right (Axler)](https://linear.axler.net/) — Rigorous  treatment of eigenvalues, singular values, and spectral theory- [Matrix Computations (Golub & Van Loan)](https://www.cs.cornell.edu/cv/GVL4/) —  The definitive reference for numerical linear algebra- [Essence of Linear Algebra (3Blue1Brown)](https://www.3blue1brown.com/topics/linear-algebra) —  Superb geometric intuition for all concepts in this notebook